In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/understanding-unets


In [2]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
import os.path as op
import time

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler
from keras_tqdm import TQDMNotebookCallback
import tensorflow as tf
from tqdm import tqdm_notebook

from learning_wavelets.datasets import im_dataset_div2k
from learning_wavelets.evaluate import psnr, ssim
from learning_wavelets.keras_utils.filters_cback import NormalizeWeights
from learning_wavelets.keras_utils.image_tboard_cback import TensorBoardImage
from learning_wavelets.keras_utils.normalisation import NormalisationAdjustment
from learning_wavelets.keras_utils.thresholding import SoftThresholding
from learning_wavelets.learned_wavelet import learnlet

Using TensorFlow backend.


In [4]:
tf.random.set_seed(1)

In [5]:
noise_std = 30
batch_size = 8
n_samples_train = 800
im_ds_train = im_dataset_div2k(mode='training', batch_size=batch_size, patch_size=256, noise_std=30)
im_ds_val = im_dataset_div2k(mode='validation', batch_size=1, patch_size=256, noise_std=30)

In [6]:
thresh = 2*noise_std/255
run_params = {
    'denoising_activation': SoftThresholding(thresh),
    'learnlet_analysis_kwargs':{
        'n_tiling': 256, 
        'mixing_details': False,        
    },
    'learnlet_synthesis_kwargs': {
        'res': True,
    },
    'n_scales': 5,
    'exact_reconstruction_weight': 0,
}
n_epochs = 250
run_id = f'learnlet_div2k_{noise_std}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

learnlet_div2k_30_1575282700


In [7]:
def l_rate_schedule(epoch):
    return max(1e-3 / 2**(epoch//25), 1e-5)
lrate_cback = LearningRateScheduler(l_rate_schedule)

In [8]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=n_epochs, save_weights_only=False)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
    profile_batch=0,
)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
tqdm_cb.on_train_batch_begin = tqdm_cb.on_batch_begin
tqdm_cb.on_train_batch_end = tqdm_cb.on_batch_end
# tqdm_cb = TQDMProgressBar()
val_noisy, val_gt = next(iter(im_ds_val))
tboard_image_cback = TensorBoardImage(
    log_dir=log_dir + '/images',
    image=val_gt[0:1],
    noisy_image=val_noisy[0:1],
)
norm_cback = NormalisationAdjustment(momentum=0.99, n_pooling=5)
norm_cback.on_train_batch_end = norm_cback.on_batch_end

W1202 11:31:40.325248 140091297724160 callbacks.py:863] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [9]:
n_channels = 1
model = learnlet(input_size=(None, None, n_channels), lr=1e-3, **run_params)
print(model.summary(line_length=114))

Model: "model"
__________________________________________________________________________________________________________________
Layer (type)                         Output Shape             Param #       Connected to                          
input_1 (InputLayer)                 [(None, None, None, 1)]  0                                                   
__________________________________________________________________________________________________________________
learnlet_analysis (LearnletAnalysis) [(None, None, None, 256) 32050         input_1[0][0]                         
__________________________________________________________________________________________________________________
normalisation (Normalisation)        (None, None, None, 256)  256           learnlet_analysis[0][0]               
                                                                            thresholding[0][0]                    
_________________________________________________________________

In [10]:
%%time
model.fit(
    im_ds_train, 
    steps_per_epoch=int(n_samples_train / batch_size), 
#     steps_per_epoch=5, 
    epochs=n_epochs,
    validation_data=im_ds_val,
#     validation_steps=int(validation_split * n_samples_train / batch_size),
    validation_steps=1,
    verbose=0,
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback, norm_cback, lrate_cback],
    callbacks=[tqdm_cb, tboard_cback, chkpt_cback, tboard_image_cback, norm_cback, lrate_cback],
    shuffle=False,
)

W1202 11:31:49.274475 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.236864). Check your callbacks.


W1202 11:33:12.894062 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.248246). Check your callbacks.


W1202 11:34:35.488249 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.247556). Check your callbacks.


W1202 11:37:21.851482 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.289366). Check your callbacks.


W1202 11:39:27.854441 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.257299). Check your callbacks.


...

W1202 14:14:27.254657 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.263908). Check your callbacks.
W1202 14:14:27.785090 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.268086). Check your callbacks.
W1202 14:14:28.241177 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.260783). Check your callbacks.
W1202 14:14:28.803516 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.268086). Check your callbacks.
W1202 14:14:29.363981 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.270153). Check your callbacks.
W1202 14:14:29.788349 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.267028). Check your callbacks.
W1202 14:14:30.334873 140091297724160 callbacks.py:244] Method (on_tra

W1202 14:14:39.428807 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.302339). Check your callbacks.
W1202 14:14:39.994362 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.295432). Check your callbacks.
W1202 14:14:40.589241 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.302339). Check your callbacks.
W1202 14:14:41.102213 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.295432). Check your callbacks.
W1202 14:14:41.630074 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.288525). Check your callbacks.
W1202 14:14:42.194314 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.287167). Check your callbacks.
W1202 14:14:42.700357 140091297724160 callbacks.py:244] Method (on_tra

W1202 14:18:15.561211 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.244103). Check your callbacks.


W1202 14:21:01.723853 140091297724160 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.262322). Check your callbacks.



CPU times: user 1d 10min 42s, sys: 5h 21min 20s, total: 1d 5h 32min 3s
Wall time: 2h 50min 40s


In [11]:
# %%time
# # overfitting trials
# data = next(iter(im_ds_train))
# val_data = next(iter(im_ds_val))
# model.fit(
#     x=data[0], 
#     y=data[1], 
# #     validation_data=val_data, 
#     batch_size=1, 
# #     callbacks=[tqdm_cb, tboard_cback, tboard_image_cback, norm_cback, lrate_cback],
#     callbacks=[tqdm_cb, tboard_cback, norm_cback,],
#     epochs=50, 
#     verbose=2, 
#     shuffle=False,
# )
# print('Original metrics')
# print(psnr(data[0].numpy(), data[1].numpy()))
# print(ssim(data[0].numpy(), data[1].numpy()))